In [4]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
import pickle
import compound_splitter
from refexps import load_data, move_level, gamerun_level, get_previous #, lex_sim
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
import compound_splitter
from scipy import stats
from nltk import word_tokenize
from string import punctuation
punct = list(punctuation)

dataset_dir = '../../En_De_Dataset/All/RefExp'
success_dir = '../../En_De_Dataset/All/Success'

def exp_len(exp, language, splits, compound):
    '''
    Extract content words by the phrase
    :param exp: referring expression tuple
    :return: set of content words of the phrase
    '''
    exp = exp[2]
    words = word_tokenize(exp, language)
    if compound == True:
        if language == 'german':
            words_nocomp = []
            for w in words:
                word = compound_splitter.split_word(w, splits)
                words_nocomp = words_nocomp + (word.split(' '))	
                words = words_nocomp
    return len(words)

def len_div(exp1, previous, language, splits, compound):
    '''
    compute dice similarity between the content words in two referring expression
    :param exp1: referring expression tuple 1
    :param exp2: referring expression tuple 2
    :return: dice similarity between the content words in the expressions; undefined if they have none
    '''        
    len_exp = exp_len(exp1, language, splits, compound)    
    previous_len = np.mean(previous)
    metric = float(previous_len - len_exp)/float(previous_len + len_exp)
    return metric if not np.isnan(metric) and metric > -1  and metric < 1 else 'undefined'

In [7]:
structure = '[english-->{move_id:(success, confid, tuple_len, tuple_lex, tuple_form}...., german ---> {}'
tuple_structure = '(general(list), p(list), e(list))'
def data_values_move(refexps_dir, success_dir, compound = True):
    splits = compound_splitter.load_dict('../../de_lower.dict')
    data_en = defaultdict(dict)
    data_de = defaultdict(dict)
    data_regression = []
    corr_data_lex = defaultdict(list)
    corr_data_form =defaultdict(list)
    success_types = set()
    confid_types = set()
    moves = 0
    moves_regr = 0
    for dialogue in listdir(refexps_dir):
        #extract dialogue type
        success_file = dialogue.replace('refexps', 'movesuccess')
        success_file = path.join(success_dir, success_file)
        if dialogue.startswith('FTT'):
            language = 'german'
        else:
            language = 'english'
        dialogue_path = path.join(refexps_dir, dialogue)
        #load referring expressions data
        dialogue_path = refexps.load_data(dialogue_path)
        success_moves = refexps.load_data(success_file)
        #move level
        dialogue_move = refexps.move_level(dialogue_path)
        #iterate over moves
        for move in dialogue_move:
            refexp_number = 0
            move_id = dialogue + str(move[0])
            success = success_moves[move[0]][0]
            confid = success_moves[move[0]][1]
            refs = move[1]
            if success == 'success':
                success = 'correct'
            if success == 'wrong':
                success = 'incorrect'
            if success == 'na':
                success = 'no_video'
            success_types.add(success)
            if confid == 'confid':
                confid = 'confident'
            if confid == 'inconf':
                condif = 'unconfident'
            if confid == 'reconf':
                confid = 'reconfirm'
            confid_types.add(confid)
            
            form_values = ([], [], [])
            len_values = ([], [], [])
            lex_values = ([], [], [])
            
            for obj in refs:
                refobj = refs[obj]
                prev_length = []
                speakers = set()
                
                divs = []
                divs_speakers = defaultdict(list)
                
                total_words = 0
                total_words_speaker = defaultdict(int)
                sameword_n = 0
                sameword_n_speaker = defaultdict(int)
                prev_words = set()
                
                total_n = len(refs[obj])
                sameform_n = 0
                sameform_n_speaker = defaultdict(int)
                total_n_speaker = defaultdict(int)
                prev_forms = set()
                for ref in refobj:
                    refexp_number += 1
                    speaker = ref[1]
                    speakers.add(speaker)
                    #LENGTH
                    if prev_length != []:
                        div = len_div(ref, prev_length, language, splits, compound)
                        if div != 'undefined':
                            divs.append(div)
                            divs_speakers[speaker].append(div)
                    len_exp = exp_len(ref, language, splits, compound) 
                    prev_length.append(len_exp)
                    #LEXICAL
                    words = refexps.lex_material(ref, language, splits, compound)
                    if prev_words != set():
                        for w in words:
                            if w in prev_words:
                                sameword_n += 1
                                sameword_n_speaker[speaker] += 1
                    else:
                        starter = speaker
                        first_words = len(words)
                    total_words_speaker[speaker] += len(words)
                    total_words += len(words)
                    for w in words:
                        prev_words.add(w)
                    #FORM 
                    form = ref[3]
                    if prev_forms != set():
                        if form in prev_forms:
                            sameform_n += 1
                            sameform_n_speaker[speaker] += 1
                    else:
                        starter = speaker
                    total_n_speaker[speaker] += 1
                    prev_forms.add(form)  
                #LENGTH

                if divs != []:
                    alignment = np.mean(divs)
                    len_values[0].append(alignment)
                for s in speakers:
                    if divs_speakers[speaker] != []:
                        alignment = np.mean(divs_speakers[speaker])
                        if s == 'p-utts':
                            len_values[1].append(alignment)
                        if s == 'e-utts':
                            len_values[2].append(alignment)
                #LEXICAL
                lexical_align = False
                lexical_align_p = False
                lexical_align_e = False
                if (total_words - first_words) != 0:
                    alignment_lex = float(sameword_n)/float(total_words - first_words)
                    lex_values[0].append(alignment_lex)
                    lexical_align = True
                for s in speakers:
                        if starter == s:
                            if (total_words_speaker[s] - first_words) != 0:
                                alignment = float(sameword_n_speaker[s])/float(total_words_speaker[s] - first_words)
                                if s == 'p-utts':
                                    lex_values[1].append(alignment)
                                    alignment_lex_p = alignment
                                    lexical_align_p = True
                                if s == 'e-utts':
                                    lex_values[2].append(alignment)
                                    lexical_align_e = True
                                    alignment_lex_e = alignment
                        else:
                            if(total_words_speaker[s]) != 0:
                                alignment = float(sameword_n_speaker[s])/float(total_words_speaker[s])
                                if s == 'p-utts':
                                    lex_values[1].append(alignment)
                                    lexical_align_p = True
                                    alignment_lex_p = alignment
                                if s == 'e-utts':
                                    lex_values[2].append(alignment)
                                    lexical_align_e = True
                                    alignment_lex_e = alignment
                                    
                #FORM
                
                if (total_n - 1) != 0:
                    alignment = float(sameform_n)/float(total_n - 1)
                    form_values[0].append(alignment)
                    if lexical_align == True:
                        corr_data_form['alignment'].append(alignment)
                        corr_data_lex['alignment'].append(alignment_lex)
                for s in speakers:
                        if starter == s:
                            if (total_n_speaker[s] - 1) != 0:
                                alignment = float(sameform_n_speaker[s])/float(total_n_speaker[s] - 1)
                                if s == 'p-utts':
                                    form_values[1].append(alignment)
                                    if lexical_align_p == True:
                                        corr_data_form['alignment p'].append(alignment)
                                        corr_data_lex['alignment p'].append(alignment_lex)
                                if s == 'e-utts':
                                    form_values[2].append(alignment)
                                    if lexical_align_e == True:
                                        corr_data_form['alignment e'].append(alignment)
                                        corr_data_lex['alignment e'].append(alignment_lex)
                        else:
                            if(total_n_speaker[s]) != 0:
                                alignment = float(sameform_n_speaker[s])/float(total_n_speaker[s])
                                if s == 'p-utts':
                                    form_values[1].append(alignment)
                                    if lexical_align_p == True:
                                        corr_data_form['alignment p'].append(alignment)
                                        corr_data_lex['alignment p'].append(alignment_lex)
                                if s == 'e-utts':
                                    form_values[2].append(alignment)
                                    if lexical_align_e == True:
                                        corr_data_form['alignment e'].append(alignment)
                                        corr_data_lex['alignment e'].append(alignment_lex)
                 
            tuple_values = (success, confid, refexp_number ,len_values, lex_values, form_values)
            moves += 1
    
            if success == 'no_video':
                pass
            else:
                moves_regr += 1
                if language == 'german':
                    data_de[move_id] = tuple_values
                else:
                    data_en[move_id] = tuple_values
    print moves, moves_regr
    data_regression = [data_de, data_en]
    return data_regression, corr_data_lex, corr_data_form
'''
move_level_data = data_values_move(dataset_dir, success_dir)
data_regression = {}
for v in move_level_data:
    data_regression[v] = move_level_data[v].values()
pickle.dump(data_regression, open('regression_lexical.p', 'w'))
'''
data_regression, corr_data_lex, corr_data_form = data_values_move(dataset_dir, success_dir)
pickle.dump(data_regression, open('data_regression.p', 'w'))

184 164


In [8]:
print data_regression

for v in corr_data_form:
    print v
    print len(corr_data_form[v]) == len(corr_data_lex[v])
    print stats.pearsonr(corr_data_form[v], corr_data_lex[v])
    if stats.pearsonr(corr_data_form[v], corr_data_lex[v])[1] < 0.05:
        print 'Significant'

[defaultdict(<type 'dict'>, {'FTT_2006-06-22pair1_refexps.p(2820, 3210)': ('correct', 'confident', 41, ([0.092757802626223698, 0.0, 0.0, -0.058493589743589744, -0.010320484961945191, 0.14285714285714285], [-0.016014930488614684, 0.0, 0.0, -0.058493589743589744, -0.072702991452991472, 0.14285714285714285], [-0.016014930488614684, 0.0, -0.072702991452991472, 0.14285714285714285]), ([0.5294117647058824, 0.0, 1.0, 0.14285714285714285, 0.5555555555555556, 0.0], [0.6428571428571429, 0.0, 1.0, 0.14285714285714285, 0.42857142857142855], [0.0, 1.0, 0.0]), ([0.8, 1.0, 1.0, 0.5, 0.9444444444444444, 1.0], [0.8571428571428571, 1.0, 1.0, 0.5, 0.9230769230769231], [0.6666666666666666, 1.0, 1.0])), 'FTT_2006-03-28pair2_refexps.p(2905, 2931)': ('not_moved', 'on_hold', 3, ([0.34920634920634919], [0.55555555555555558], [0.55555555555555558]), ([0.5], [0.5], []), ([1.0], [1.0], [1.0])), 'FTT_2006-03-27Pair1_refexps.p(3427, 3443)': ('correct', 'unconfident', 1, ([], [], []), ([], [], []), ([], [], [])), 'F